In [24]:
import pandas as pd
from example.discretizer.discretizer_d0  import AVDiscretizer
from example.discretizer.discretizer_d1  import AVDiscretizerD1
from example.discretizer.utils import Action
import matplotlib.pyplot as plt
from pathlib import Path
import pgeon.policy_graph as PG
from example.environment import SelfDrivingEnvironment
from nuscenes import NuScenes

In [ ]:
#path where you stored the copy of the nuScenes dataset.
DATAROOT = Path('/home/saramontese/Desktop/MasterThesis/example/dataset/data/sets/nuscenes')

#MINI
nuscenes = NuScenes('v1.0-mini', dataroot=DATAROOT)

#FULL
#nuscenes = NuScenes('v1.0-trainval', dataroot=DATAROOT)

The dataset contains the following columns for each state of the self-driving car:

- instance_token: A unique identifier for the vehicle instance.
- translation: The vehicle's position in 3D space (x, y, z coordinates).
- yaw: The orientation of the vehicle around the vertical axis (rotation angle).
- velocity: The vehicle's speed in a given direction.
- acceleration: The change in velocity over time.
- heading_change_rate: The rate of change of the vehicle's direction.
- timestamp: The time at which the state was recorded.
- scene_token: A unique identifier for the scenario or environment the vehicle is in
- detected_objects: detected objects in the surrounding of the vehicle


Load the CSV data into a DataFrame


In [ ]:

dtype_dict = {
    'modality': 'category',  # for limited set of modalities, 'category' is efficient
    'scene_token': 'str',  
    'timestamp': 'str',  # To enable datetime operations
    'rotation': 'object',  # Quaternion (lists)
    'x': 'float64',
    'y': 'float64',
    'z': 'float64',
    'yaw': 'float64',  
    'velocity': 'float64',
    'acceleration': 'float64',
    'heading_change_rate': 'float64',
    'delta_local_x': 'float64',
    'delta_local_y': 'float64'}
df = pd.read_csv(DATAROOT / 'train_v1.0-mini_lidar_3.csv', dtype=dtype_dict, parse_dates=['timestamp'])
city ='singapore-onenorth'  #'boston-seaport'#boston-seaport'#
df = df[df['location'] == city]
#df['detect_CAM_FRONT'] = df['detect_CAM_FRONT'].apply(lambda x: ast.literal_eval(x))
#df['detect_CAM_BACK'] = df['detect_CAM_BACK'].apply(lambda x: ast.literal_eval(x))
df.head()

## Trajectory of AV in a Scene

Let's do an example of computing trajectory of a vehicle in a scene. We then check if the rendering of the scene match the computed trajectory.

List of scenes

In [27]:
df['scene_token'].unique()

array(['cc8c0bf57f984915a77078b10eb33198'], dtype=object)

Select a scene and test the  algorithm


In [ ]:
scene_test = df['scene_token'].unique()[0]
example_scene_df = df[df['scene_token']==scene_test]
example_scene_df.head()

Compute the trajectoy of the scene

In [ ]:
env = SelfDrivingEnvironment(city,dataroot=DATAROOT)
av_discretizer = AVDiscretizer(env)
pg = PG.PolicyGraph(env, av_discretizer)
pg = pg.fit(df, update=False, verbose=True)

To render the scene (video):

In [30]:
#nuscenes.render_scene(scene_test)

In [31]:
environment = SelfDrivingEnvironment(city='all')
discretizer = AVDiscretizerD1(environment, obj_discretization='multiple', vel_discretization='multiple')
nodes_path = f'example/dataset/data/policy_graphs/PG_mini_Call_D1c_Wall_Tall_nodes.csv'
edges_path = f'example/dataset/data/policy_graphs/PG_mini_Call_D1c_Wall_Tall_edges.csv'
pg = PG.PolicyGraph.from_nodes_and_edges(nodes_path, edges_path, environment, discretizer)


# Policy Graph Generation

In [32]:
print(f'Number of nodes: {len(pg.nodes)}')
print(f'Number of edges: {len(pg.edges)}')

Number of nodes: 161
Number of edges: 258


In [33]:
arbitrary_state = list(pg.nodes)[9]

print(arbitrary_state)
print(f'  Times visited: {pg.nodes[arbitrary_state]["frequency"]}')
print(f'  p(s):          {pg.nodes[arbitrary_state]["probability"]:.3f}')

(PedestrianNearby(1-3), IsTwoWheelNearby(NO), BlockProgress(START), LanePosition(RIGHT), NextIntersection(NONE), Velocity(STOPPED), Rotation(RIGHT), IsStopSignNearby(NO), IsZebraNearby(YES), IsTrafficLightNearby(YES), FrontRightObjects(1-3)), FrontLeftObjects(1-3)))
  Times visited: 5
  p(s):          0.015


In [34]:
arbitrary_edge = list(pg.edges)[20]

print(f'From:    {arbitrary_edge[0]}')
print(f'Action:  {arbitrary_edge[2]}')
print(f'To:      {arbitrary_edge[1]}')
print(f'  Times visited:      {pg[arbitrary_edge[0]][arbitrary_edge[1]][arbitrary_edge[2]]["frequency"]}')
print(f'  p(s_to,a | s_from): {pg[arbitrary_edge[0]][arbitrary_edge[1]][arbitrary_edge[2]]["probability"]:.3f}')

From:    (PedestrianNearby(0), IsTwoWheelNearby(YES), BlockProgress(INTERSECTION), LanePosition(RIGHT), NextIntersection(NONE), Velocity(LOW), Rotation(RIGHT), IsStopSignNearby(NO), IsZebraNearby(YES), IsTrafficLightNearby(YES), FrontRightObjects(1-3)), FrontLeftObjects(0)))
Action:  9
To:      (PedestrianNearby(0), IsTwoWheelNearby(YES), BlockProgress(INTERSECTION), LanePosition(RIGHT), NextIntersection(NONE), Velocity(LOW), Rotation(RIGHT), IsStopSignNearby(NO), IsZebraNearby(YES), IsTrafficLightNearby(YES), FrontRightObjects(4+)), FrontLeftObjects(0)))
  Times visited:      1
  p(s_to,a | s_from): 0.500


### Using PG-based policies

In [35]:
from pgeon.policy_graph import PGBasedPolicy, PGBasedPolicyMode, PGBasedPolicyNodeNotFoundMode

from example.discretizer.utils import PedestrianNearby, BlockProgress, IsTwoWheelNearby, IsTrafficLightNearby,IsZebraNearby,IsStopSignNearby, FrontLeftObjects, FrontRightObjects, Action, LanePosition, BlockProgress, NextIntersection, Velocity, Rotation

from pgeon.discretizer import  Predicate

In [36]:
policy_mode = PGBasedPolicyMode.STOCHASTIC
node_not_found_mode = PGBasedPolicyNodeNotFoundMode.FIND_SIMILAR_NODES

policy = PGBasedPolicy(pg, policy_mode,node_not_found_mode )
pg._normalize()
print(f'Policy Graph with {policy_mode} and {node_not_found_mode}')
print(f'PG number of nodes: {len(policy.pg.nodes)}')
print(f'PG number of edges: {len(policy.pg.edges)}')
        


initial_state = list(pg.nodes)[1]  
#final_state = list(pg.nodes)[5]
action = policy.act(initial_state)

action



Policy Graph with PGBasedPolicyMode.STOCHASTIC and PGBasedPolicyNodeNotFoundMode.FIND_SIMILAR_NODES
PG number of nodes: 161
PG number of edges: 258


9

In [37]:
str_obs = discretizer.state_to_str(initial_state)
str_obs

'PedestrianNearby(0) IsTwoWheelNearby(YES) BlockProgress(INTERSECTION) LanePosition(RIGHT) NextIntersection(NONE) Velocity(LOW) Rotation(RIGHT) IsStopSignNearby(NO) IsZebraNearby(YES) IsTrafficLightNearby(YES) FrontRightObjects(4+)) FrontLeftObjects(0))'

Question 1

In [38]:
possible_actions = pg.question1(initial_state, verbose=True)

I will take one of these actions:
	-> BRAKE_TURN_RIGHT 	Prob: 100.0 %


Question 2

In [ ]:
action_id = AVDiscretizer.get_action_id(Action.BRAKE)
best_states = pg.question2(action_id, verbose=True)

Question 3

In [ ]:
state = tuple(( Predicate(PedestrianNearby, [PedestrianNearby(1, 'multiple')]),
                Predicate(IsTwoWheelNearby, [IsTwoWheelNearby.YES]),
                Predicate(BlockProgress, [BlockProgress.END]),
                
                Predicate(LanePosition, [LanePosition.RIGHT]),
                Predicate(NextIntersection, [NextIntersection.STRAIGHT]),
                Predicate(Velocity, [Velocity.STOPPED]),
                Predicate(Rotation, [Rotation.FORWARD]),
                Predicate(IsStopSignNearby, [IsStopSignNearby.NO]), 
                Predicate(IsZebraNearby,[IsZebraNearby.NO]),
                Predicate(IsTrafficLightNearby, [IsTrafficLightNearby.NO]),
                Predicate(FrontRightObjects,[FrontRightObjects(1, 'multiple')]),
                Predicate(FrontLeftObjects,[FrontLeftObjects(1, 'multiple')])))
#state_2 = av_discretizer.str_to_state(av_discretizer.state_to_str(state))

action = policy.act(state)

In [18]:
policy_mode= PGBasedPolicyMode.GREEDY
node_not_found_mode = PGBasedPolicyNodeNotFoundMode.FIND_SIMILAR_NODES #PGBasedPolicyNodeNotFoundMode.RANDOM_UNIFORM, 
action = Action.BRAKE
action_id = AVDiscretizer.get_action_id(action)

policy_2 = PGBasedPolicy(pg, mode=policy_mode,  node_not_found_mode=node_not_found_mode)
print(f'Policy Graph with {policy_mode} and {node_not_found_mode}')
print()
print(f'Supposing I was in {state}, '
    f'if I did not choose to {action} was due to...')
policy.pg._normalize() #do i need to normalize before this step?
counterfactuals = policy_2.pg.question3(state, action_id, greedy= True, verbose=True)
        

Policy Graph with PGBasedPolicyMode.GREEDY and PGBasedPolicyNodeNotFoundMode.FIND_SIMILAR_NODES

Supposing I was in (PedestrianNearby(1-3), IsTwoWheelNearby(YES), BlockProgress(END), LanePosition(RIGHT), NextIntersection(STRAIGHT), Velocity(STOPPED), Rotation(FORWARD), IsStopSignNearby(NO), IsZebraNearby(NO), IsTrafficLightNearby(NO), FrontRightObjects(YES), FrontLeftObjects(YES)), if I did not choose to Action.BRAKE was due to...
***********************************************
* Why did not you perform X action in Y state?
***********************************************
PGBasedPolicyNodeNotFoundMode.RANDOM_UNIFORM
I would have chosen: 1
I would have chosen 5 under the following conditions:
	I don't know where I would have ended up


# Policy Graph Evaluation and Comparison

Ideal situation: knowing the current state unqeuivocally determines the following action and state. To evaluate this we use Entropy.

Entropy Interpretation: level of uncertainty of a variable (check PG_position_paper).

- H_a(s): 0 --> state s perfectly determines the following action a.
Too simple graphs with few nodes show larger action uncertainty, making output less reliable.

- H_w(s): 0 --> state s perfectly determines the following state s'.

- H(s): 0 --> state s perfectly determines the following pais of s', a.

## Improve Policy

In [19]:
#path_pg_data = Path('/home/saramontese/Desktop/MasterThesis')

#sub_pg = PG.PolicyGraph.from_nodes_and_edges(str(path_pg_data / 'nuscenes_nodes.csv'), str(path_pg_data / 'nuscenes_edges.csv'), env, av_discretizer  )

In [20]:
#initial_state = example_scene_df.iloc[0][['x', "y", "velocity", "yaw"]].values
#final_state = example_scene_df.iloc[-1][['x', "y", "velocity", "yaw"]].values
#print('initial scene state: ', initial_state)
#print('final scene state: ', final_state)

In [21]:
#sub_agent = PGBasedPolicy(sub_pg, mode=PGBasedPolicyMode.GREEDY)
#reward = env.compute_total_reward(sub_agent, initial_state, final_state, max_steps=100)
#print('scene reward before improvement: ', reward)

In [22]:
#reward = env.compute_total_reward(pg)
#print('Total PG reward before improvement: ', reward)

In [23]:
#pg.policy_iteration(sub_pg)
#print('scene reward after improvement: ', env.compute_total_reward(pg))
